# Bike sharing (Kaggle) (Moritz)
- https://www.kaggle.com/c/184702-tu-ml-ws-18-bike-sharing
- large samples (train = 8690), small dimension (15)
- attribute characteristics: numeric, date?

## Preprocessing
- scale (standardize)
- drop date-string col ('dteday')

### Linear Regression
Score created from training data without(!) train-test split.
- with and without preprocessing
- with all samples: 
    - <1 s
    - R^2: 0.38403
    - RMSE: 140.25758
    - Kaggle: 143.91026

### SVR
- without preprocessing
- with all samples
    - few minutes
    - C: 1.0, kernel: linear, epsilon: 0.30000000000000004, gamma: auto 
    - RMSE: 147.88896
    - Kaggle: 152.19112
- with preprocessing
- with all samples: 
    - few minutes   
    - C: 1.0, kernel: linear, epsilon: 0.5, gamma: auto 
    - RMSE: 146.11651
    - Kaggle: 150.71520

### Gradient Boosted Decision Tree
- with preprocessing
- with 50 samples: 
    - 13.559 s
    - {'max_depth': 5, 'min_samples_split': 15, 'n_estimators': 350}
    - RMSE: 89.9571
    - Kaggle: 143.91026
- with 150 samples: 
    - 25.025 s
    - {'max_depth': 5, 'min_samples_split': 15, 'n_estimators': 350}
    - RMSE: 118.99678
    - Kaggle: 128.42365
- with 500 samples: 
    - 74.803 s
    - {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 350}
    - RMSE: 78.88731
    - Kaggle: 128.39004
- with 2000 samples:
    - 431.691 s
    - {'max_depth': 10, 'min_samples_split': 15, 'n_estimators': 350}
    - RMSE: 54.24448
    - Kaggle: 54.09653
- with all samples:
    - 4361.643 s
    - {'max_depth': 10, 'min_samples_split': 15, 'n_estimators': 350}
    - RMSE: 43.12057
    - Kaggle: 43.16327
    
### AutoML 
- without scaling
- with all samples
    - minutes? (max 600 s)
    - XGBoost_3_AutoML_20190106_135032
    - RMSE: 40.08211
    - Kaggle: 39.61732
    
    
- more models (60), more time (7200 s)
    - ?
    - StackedEnsemble_AllModels_AutoML_20190106_155951
    - RMSE: 38.94180
    - Kaggle: __38.10583__
    
    
- with preprocessing
- with all samples
    - minutes (max 600 s)
    - XGBoost_3_AutoML_20190106_144525
    - RMSE: 40.24830
    - Kaggle: 39.70656

In [9]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import datetime as dt
%run './base.ipynb'

In [10]:
# read train data and drop date
train = pd.read_csv('./data/bike_sharing_kaggle/bikeSharing.shuf.train.csv').drop(['dteday'], axis=1)
train.set_index(['id'], inplace=True)
# extract, then drop 'cnt' col
train_target = train[['cnt']]
train.drop(['cnt'], axis='columns', inplace=True)

# read test data and drop date
test = pd.read_csv('./data/bike_sharing_kaggle/bikeSharing.shuf.test.csv').drop(['dteday'], axis=1)
test.set_index(['id'], inplace=True)

# scale train
#train_s, test_s = scale_data(train, test)

#X_train = train_s
#y_train = train_target
#X_test = test_s

X_train = train
y_train = train_target
X_test = test

#display(train)
#display(X_train.shape)

In [3]:
# Linear Regression
# predict on X_train (to replace missing MSE in LinearRegression)
reg = linear_reg(X_train, y_train, X_train, y_train)
result = pd.DataFrame(reg.predict(X_test), columns=['cnt'])

# join id col
result = pd.concat([X_test.reset_index()[['id']], result], axis='columns')

# Save result
filename = f'''lr_{dt.datetime.now()}.csv'''

result.to_csv('./predictions/bike_sharing_kaggle/' + filename, sep = ",", index=False)
print(f'''Saved as {filename}''')

#display(result)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


R^2 value for model: 0.38403
Predict:
RMSE: 140.25758
R^2 Score: 0.38403
Saved as lr_2019-01-06 13:34:13.249858.csv


In [4]:
# SVR
# params
param_grid = {
    'C': np.linspace(.2,1,5),
    'kernel': ['linear'],#, 'rbf', 'sigmoid', 'poly'], # poly very slow
    'epsilon': np.linspace(0,.5,6),
    'gamma': ['auto']
}

# run grid search
gs = run_svr(X_train, y_train.values.ravel(), cv=5, param_grid=param_grid)

# predict
result = pd.DataFrame(gs.best_estimator_.predict(X_test), columns=['cnt'])

# join id col
result = pd.concat([X_test.reset_index()[['id']], result], axis='columns')
#display(result)

# Create SVR filename
filename = f'''svr_'''\
           f'''C-{gs.best_estimator_.C}_'''\
           f'''k-{gs.best_estimator_.kernel}_'''\
           f'''e-{gs.best_estimator_.epsilon}_'''\
           f'''g-{gs.best_estimator_.gamma}_'''\
           f'''{dt.datetime.now()}.csv'''

result.to_csv('./predictions/bike_sharing_kaggle/' + filename, sep = ",", index=False)
print(f'''Saved as {filename}''')

GridSearch initializing...
SVR model in training...
MSE: 21871.14445, RMSE: 147.88896, 
C: 1.0, kernel: linear, epsilon: 0.30000000000000004, gamma: auto 
Saved as svr_C-1.0_k-linear_e-0.30000000000000004_g-auto_2019-01-06 13:43:58.951959.csv


In [21]:
# Gradient Boosted Decision Tree
param_fix = {
    'learning_rate': .01, 
    'loss': 'ls'
}

param_grid = {
    'n_estimators': (1, 10, 100, 200, 350),# 500), 
    'max_depth': (1, 5, 10, 25),# 50), 
    'min_samples_split': (2, 5, 15),# 50)
}

num_samples = 500
#X = X_train.iloc[:num_samples, :]
#y = y_train.iloc[:num_samples, :].values.ravel()

X = X_train
y = y_train.values.ravel()

gs = run_boosted_tree(X, y, [], [], param_fix=param_fix, cv=10, param_grid=param_grid)

#plot_scores(gbt.cv_results_)
#plot_training_deviance(gbt, test_data, test_target)

# predict
result = pd.DataFrame(gs.best_estimator_.predict(X_test), columns=['cnt'])

# join id col
result = pd.concat([X_test.reset_index()[['id']], result], axis='columns')
#display(result)

# Create SVR filename
filename = f'''gbdtree_'''\
           f'''ne-{gs.best_estimator_.n_estimators}_'''\
           f'''md-{gs.best_estimator_.max_depth}_'''\
           f'''mss-{gs.best_estimator_.min_samples_split}_'''\
           f'''{dt.datetime.now()}.csv'''

result.to_csv('./predictions/bike_sharing_kaggle/' + filename, sep = ",", index=False)
print(f'''Saved as {filename}''')

GridSearch initializing...
GradientBoostedRegressor model in training...
GradientBoostedRegressor model selected and fitted in 4361.643 s

MSE: 1859.38335, RMSE: 43.12057
Best parameters selected by GridSearch: {'max_depth': 10, 'min_samples_split': 15, 'n_estimators': 350}
Saved as gbdtree_ne-350_md-10_mss-15_2019-01-02 01:53:02.486364.csv


In [11]:
# AutoML
# reset indices to id col
X_train_r = X_train.reset_index()
#display(X_train_r)
y_train_r = y_train.reset_index()
#display(y_train_r.info())

# create train-test-split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train_r, y_train_r, random_state=100)
train = pd.merge(X_train1, y_train1, on='id')

# drop id cols from test set
X_test2 = X_test1.drop(['id'], axis=1)
y_test2 = y_test1.drop(['id'], axis=1)

y_name = 'cnt'
aml = run_autoML_moritz(train, y_name, X_test2, y_test2, 
                        max_models=30, max_runtime=300)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,5 hours 40 mins
H2O cluster timezone:,Europe/Vienna
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.1
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_Moritz_rnh332
H2O cluster total nodes:,1
H2O cluster free memory:,2.184 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
AutoML training performed in 308.85472106933594 s.


'AutoML Leaderboard'

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_3_AutoML_20190106_191613,1606.58,40.0821,1606.58,25.6245,nan
XGBoost_2_AutoML_20190106_191613,1623.96,40.2984,1623.96,25.7851,nan
GBM_4_AutoML_20190106_191613,1662.89,40.7786,1662.89,25.8958,nan
StackedEnsemble_AllModels_AutoML_20190106_191613,1663.78,40.7894,1663.78,25.9587,nan
GBM_3_AutoML_20190106_191613,1686.72,41.0697,1686.72,26.0725,nan
GBM_2_AutoML_20190106_191613,1713.47,41.3941,1713.47,26.4211,nan
GBM_1_AutoML_20190106_191613,1754.87,41.8911,1754.87,26.8181,nan
XGBoost_1_AutoML_20190106_191613,1861.26,43.1424,1861.26,27.9436,nan
XGBoost_grid_1_AutoML_20190106_191613_model_1,1881.48,43.3761,1881.48,27.9726,nan
StackedEnsemble_BestOfFamily_AutoML_20190106_191613,2145.66,46.3213,2145.66,30.1142,nan


In [7]:
print(aml.leader)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20190106_155951
No model summary for this model


ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 296.99413300418286
RMSE: 17.23351771996022
MAE: 10.945189812719754
RMSLE: NaN
R^2: 0.9908355293330577
Mean Residual Deviance: 296.99413300418286
Null degrees of freedom: 6516
Residual degrees of freedom: 6491
Null deviance: 211197224.05462587
Residual deviance: 1935510.7647882598
AIC: 55654.36845100865

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 1516.4638065123465
RMSE: 38.94180024745064
MAE: 24.150401962797083
RMSLE: NaN
R^2: 0.9532058497867152
Mean Residual Deviance: 1516.4638065123465
Null degrees of freedom: 6516
Residual degrees of freedom: 6489
Null deviance: 211294516.62786394
Residual deviance: 9882794.627040962
AIC: 66283.84215410653



In [8]:
# create predictions for test data
X_test_h2o = h2o.H2OFrame(X_test.reset_index())
result = aml.predict(X_test_h2o)
#result.head(rows=result.nrows)

result_df = result.as_data_frame()
result_df[['id']] = X_test.reset_index()[['id']]
result_df.rename({'predict': 'cnt'}, axis=1, inplace=True)
#display(result_df)

# save to file
filename = f'''autoML_'''\
           f'''{dt.datetime.now()}.csv'''

result_df.to_csv('./predictions/bike_sharing_kaggle/' + filename, sep = ",", index=False)
print(f'''Saved as {filename}''')

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
Saved as autoML_2019-01-06 18:57:48.786743.csv


In [5]:
# shutdown h2o cluster
h2o.cluster().shutdown()

AttributeError: 'NoneType' object has no attribute 'shutdown'